# Feature Transformation

#### Transform all features using rolling with the parameter 10. Tip: store the transformed dataframes in new pickle files. You never know if you need them later

#### What does the parameter (.rolling(10)) mean?

#### Make the classification again (same classes as above) as previously, with KNN or Decision Trees. Compare the results with previous, untransformed features. Is the result improved?

#### Try with a larger parameter, for example, 50. Is the result improved?